In [23]:
from copy import deepcopy

class XO_Board() :

  def __init__(self,board = None):
    # define players
    self.first_player =     'x'
    self.second_player =     'o'
    self.empty_place = '-'
    self.user_input = None

    #define the board position
    self.position = {}

    # init the board
    self.init_board()

    # create a copy of previous board state 
    if board :
      self.__dict__ = deepcopy(board.__dict__)

  # initialize the board
  def init_board(self):
    # loop over board rows
    for row in range(3):
      # loop over board columns
      for col in range(3):
        # set every square to empty square
        self.position[row,col] = self.empty_place

  # implement make move
  def make_move(self,row,col):
    # create new board instance that inherits from the current state
    board = XO_Board(self)

    # make move 
    board.position[row,col] = self.first_player

    #swap players
    (board.first_player , board.second_player) = (board.second_player , board.first_player)

    # return new board state
    return board
  

  # get whether the game is draw
  def is_draw(self):
    # loop over board
    for row , col in self.position:
      # empty square is available
      if self.position[row,col] == self.empty_place:
        return False

    # by default return True
    return True

  # get whether the game is win
  def is_win(self):
    # vertical sequence detection
    # loop over board columns
    for col in range(3):
      # define the winning sequence list
      winning_sequence = []

      # loop over board rows
      for row in range(3):
        if self.position[row , col] == self.second_player:
          # update winning sequence
          winning_sequence.append((row,col)) 
        # if we have 3 elements in the row 
        if len(winning_sequence) == 3 :
          return True

    # horizontal sequence detection
    # loop over board rows
    for row in range(3):
      # define the winning sequence list
      winning_sequence = []

      # loop over board columns
      for col in range(3):
        # if found the same next element in the row
        if self.position[row , col] == self.second_player :
          # update winning sequence
          winning_sequence.append((row,col)) 
        # if we have 3 elements in the row 
        if len(winning_sequence) == 3 :
          return True

   
    # 1st diagonal sequence detection
    
    # define the winning sequence list
    winning_sequence = []
    # loop over board rows
    for row in range(3):
      # init column
      col = row
      if self.position[row , col] == self.second_player :
        # update winning sequence
        winning_sequence.append((row,col)) 
      # if we have 3 elements in the row 
      if len(winning_sequence) == 3 :
        return True

    # 2nd diagonal sequence detection  
    # define the winning sequence list
    winning_sequence = []
    # loop over board rows
    for row in range(3):
      # init column
      col = 3-row-1
      if self.position[row , col] == self.second_player :
        # update winning sequence
        winning_sequence.append((row,col)) 
      # if we have 3 elements in the row 
      if len(winning_sequence) == 3 :
        return True


    return False

  # generate legal moves
  def generate_moves(self):
    actions = []
    # loop over board rows
    for row in range(3):
      # loop over board columns
      for col in range(3):
        # make sure the current position is empty
        if self.position[row,col] == self.empty_place :
          # append available actions/board state to action list
          actions.append(self.make_move(row,col))

    # return the list of available actions (board class instances)
    return actions

  # main game loop 
  def game(self,num_iter):
    print('   \nWelcome to X O Game:')
    print('===================================')
    user_input_1 = ''
    while True:
        print(' \nPlease Choose if you want to play with x or o  ? \n')
        user_input_1 = input()
        if user_input_1 == 'x':
          self.first_player = 'x'
          self.second_player = 'o'
          break
        elif user_input_1 == 'o':
          self.first_player = 'o'
          self.second_player = 'x' 
          break
        elif user_input_1 == 'exit':
          return
        else:
            continue
    if  user_input_1 == 'x' :       
         print('\nYou will play with x , and the Agent will play with o\n')
    else:
         print('\nYou will play with o , and the Agent will play with x\n')   

    print('enter your move for example : [x,y] = 1,2 where 1 is row and 2 is col,  or type "exit" to quit the game')

    self.user_input = user_input_1
    print(self)

    # create monti carlo tree search instance
    mcts = MCTS()

    # game loop
    while True :
      # user input
      user_input = input('> ')

      if user_input == 'exit':
        break

      # skipp empty input
      if user_input == '':
        continue
      try:
        # parse user input ===> ex : format for move [col,row] = (1,3)
        row = int(user_input.split(',')[0]) - 1
        col = int(user_input.split(',')[1]) - 1

        # check if the move is legal or not
        if self.position[row,col] != self.empty_place:
          print('Illegal move !!!')
          continue

        # make move on board
        self = self.make_move(row,col)
        # print board
        print('-------------------------You Played------------------------ :)\n')
        print(self) 


        # search for the best move
        best_move = mcts.search(self,num_iter)


        # make AI Agent move ....
        # legal moves available
        try:
           self = best_move.board
        except:
           pass   


        # print board
        print('------------------------Agent Played----------------------- :)\n')
        print(self) 

        # check the game state
        if self.is_win():
          print(' Player "%s" has won the game \n' % self.second_player )
          break

        elif self.is_draw():
          print('Oooh We have a draw \n')
          break   


      except Exception as e:
        print('Error: ' , e)
        print('Illegal command !!!')
        print('enter your move for example : [x,y] = 1,2 where 1 is row and 2 is column,  or type "exit" to quit the game')


  # print the board state
  def __str__(self):
    # define board string representation
    board_str = ''
    # loop over board rows
    for row in range(3):
      # loop over board columns
      for col in range(3):
        board_str += ' %s' % self.position[row,col]

      # print new line 
      board_str += '\n'

    # side to move 
    if self.first_player == self.user_input:
      board_str = '\n---------------------------\n "Your Turn ==>  "%s" to go"   \n---------------------------\n\n' %self.user_input + board_str
    
    else:
      board_str = '\n---------------------------\n "Agent Turn"    \n---------------------------\n\n' + board_str

    return board_str
   


# Monti Carlo Tree Search:

In [24]:
import math
import random

In [25]:
# Tree node class 
class TreeNode():
  def __init__(self,board,parent):
    self.board = board

    # check if the node is terminal
    if self.board.is_win() or self.board.is_draw():
      # that means the game is over
      self.is_terminal = True 
    else :
      # we have a non terminal node
      self.is_terminal = False  

    # initialise is fully expanded flag
    self.is_fully_expanded = self.is_terminal
    # initialise parent node if available 
    self.parent = parent  
    # initialize the number of node visits
    self.visits = 0

    # initialize the total score of the node
    self.score =  0

    # initialize the current node children
    self.children = {}  


In [26]:
class MCTS():
  # search for the best move in the current position
  def search(self,initial_state,num_iter):
    # create root node
    self.root = TreeNode(initial_state,None)

    # Do iterations
    for iteration in range(num_iter):
      # select a node (selection phase)
      node = self.select(self.root)

      # score current node (simulation phase)
      score = self.simulate_game(node.board)
      # backpropagate the number of visits and the score
      self.backpropagate(node,score)

    # pick up the best move in the current position
    try:
      return self.get_best_move(self.root,0)
    except:
      pass  

  # select most promising node
  def select(self,node):
    # make sure that it is not a terminal node
    while not node.is_terminal:
      # case where the node is fully expanded
      if node.is_fully_expanded:
        node = self.get_best_move(node,2)
      # case where the node is not fully expanded
      else:
        # expand the nodes
        return self.expand(node)
    return node


  # expand node
  def expand(self,node):
    # generate legal moves for the given node
    states = node.board.generate_moves()
    # loop over generated 
    for state in states:
      # make sure that the current node is not present in child nodes 
      if str(state.position) not in node.children:
        # create new node
        new_node = TreeNode(state,node)
        # add child node to parent's node children
        node.children[str(state.position)] = new_node

        # check if node is fully expanded
        if len(states) == len(node.children):
          node.is_fully_expanded = True
        # return newly created node
        return new_node  


  # simulate the game by making random moves until reach the end of the game
  def simulate_game(self,board):
    # make random moves for both sides until terminal state is reached
    while not board.is_win():
      # try to make a move
      try:
        # make the move on board
        board = random.choice(board.generate_moves())
        
      except:
        # return a draw score    
        return 0
        
    # return the score from player x perspective
    if board.second_player == board.user_input:
      return 1
    else:
      return -1    


  #  backpropagate   
  def backpropagate(self,node,score):
    # update node visit count and score up to root node
    while node is not None:
      # update node's visits
      node.visits += 1
      # update node's score
      node.score += score
      # set node to parent
      node = node.parent

  # select the best node based on UCB1 formula
  def get_best_move(self,node,exploration_factor):
    # define best score and best moves
    best_score = float('-inf')
    best_moves = []
    # loop over node's children
    for child in node.children.values():
      # define current player
      if child.board.second_player == child.board.user_input: 
        current_player = 1
      else:   
        current_player = -1

      # use UCB1 formula to get the move score
      move_score = current_player * child.score / child.visits + exploration_factor * math.sqrt(math.log(node.visits/child.visits))

      #better move has been found 
      if move_score > best_score:
        best_score = move_score
        best_moves = [child]

      # move score is equal to the best score  
      elif move_score == best_score :
        best_moves.append(child)

    # return one of the best moves randomly
    return random.choice(best_moves)

In [ ]:
if __name__ == '__main__':
  # create board instance 
  board = XO_Board()
  # create mcts
  mcts = MCTS()
  # start game loop 
  board.game(2000)